In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict, Dict, List
from dotenv import load_dotenv
import requests
load_dotenv()

True

In [2]:
llm = HuggingFaceEndpoint(
    endpoint_url="openai/gpt-oss-20b",
    task="text-generation",
    )

In [3]:
Model = ChatHuggingFace(llm=llm)

In [12]:
# Defining State 
class PaperInfo(TypedDict):
    topic: str
    offset: int
    top_search: int
    title: List[str]
    abstract: List[str]
    url: List[str]
    citationCount: List[int]
    result: str
    

In [13]:
# Function to get papers from Semantic Scholar API

def get_papers(Info :PaperInfo) -> PaperInfo:
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": Info['topic'],
        "fields": "title,url,abstract,citationCount",
        "limit": Info.get('top_search', 3),
        "offset": Info.get('offset', 0)
    }
    response = requests.get(url, params=params)
    # response.raise_for_status() 
    data = response.json()

    
    for paper in data.get("data", []):
        Info["abstract"].append( paper.get("abstract"))
        Info["title"].append(paper.get("title"))
        Info["url"].append(paper.get("url"))    
        Info["citationCount"].append(paper.get("citationCount"))
    return Info

In [14]:
# Finail answer Drafting function
def draft_answer(Info: PaperInfo) -> PaperInfo:
    prompt = f"Using the following papers, draft the summerization of each paper '{Info['topic']}'.\n\n"
    for i in range(len(Info['title'])):
        prompt += f"Title: {Info['title'][i]}\nAbstract: {Info['abstract'][i]}\nURL: {Info['url'][i]}\nCitations: {Info['citationCount'][i]}\n\n"
    prompt += "Summarize the key points from these papers in a concise manner.\n\n Title: \n Citations: \n Abstract Summery: \n URL: \n "
    
    response = Model.invoke(prompt)
    Info['result'] = response.content
    return Info

In [15]:
# Graph Definition
graph = StateGraph(PaperInfo)

# adding notes
graph.add_node("get_papers", get_papers)
graph.add_node("draft_answer", draft_answer)

# adding edges
graph.add_edge(START, "get_papers")
graph.add_edge("get_papers", "draft_answer")
graph.add_edge("draft_answer", END)

# Completing the graph
research_paper_graph = graph.compile()

In [16]:
def get_paper(Info :PaperInfo) -> None:
    url = "http://api.semanticscholar.org/graph/v1/paper/search/bulk"
    params = {
        "query": Info['topic'],
        "fields": "title,url,abstract,citationCount",
        "limit": 3,
        "offset": Info.get('offset', 0)
    }
    response = requests.get(url, params=params)
    print(response,'Hi Swarup')

In [ ]:
# example input
initial_state: PaperInfo = {
    "topic": "Artificial Intelligence in Healthcare",
    "offset": 0,
    "top_search": 4,
    "title": [],
    "abstract": [],
    "url": [],
    "citationCount": [],
    "result": ""
} 


In [ ]:
final_state = research_paper_graph.invoke(initial_state)
print(final_state['result'])

"final_state = research_paper_graph.invoke(initial_state)\nprint(final_state['result'])"